In [5]:
search_for = 122
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002355337142944336
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 436374755
type: [1, 1, 1, 1, 122] 122
cases of this type: 221533456
100000 15.770075459196242
200000 30.060487759724836
300000 32.63028792826636
400000 32.988077658004194
500000 33.55358353563437
600000 32.24258045009456
700000 31.537932014743276
800000 32.202712877669065
900000 26.47526707386022
1000000 17.893943090673663
1100000 32.087823706574355
1200000 32.44300923243453
1300000 32.11800887608416
1400000 33.95578138799465
1500000 32.22828959154449
1600000 32.238507202688716
1700000 32.02877197421193
1800000 30.34975806364398
1900000 14.425970529812465
2000000 31.529436196906623
2100000 32.09706821762089
2200000 30.911936213261573
2300000 32.51416432173583
2400000 31.092715217976753
2500000 31.048788377747456
2600000 30.479896642433378
2700000 29.96831415050656
2800000 13.62438513647125
2900000 30.25594453199772
3000000 30.66

29600000 29.321183004605086
29700000 29.530020315406727
29800000 27.06640220014407
29900000 26.678991139548383
30000000 12.453256782956052
30100000 27.32666389338049
30200000 26.48629155375891
30300000 28.825445076886353
30400000 29.22656807426634
30500000 30.730214679548205
30600000 30.111210319711873
30700000 16.196851156572166
30800000 30.070931873668005
30900000 30.765358430987494
31000000 32.34264600835956
31100000 31.01331273508245
31200000 16.581511240184774
31300000 32.30253366671635
31400000 31.30725213690126
31500000 31.328163429387704
31600000 29.176830328363977
31700000 28.76065917074412
31800000 15.635320022008548
31900000 23.70055040527285
32000000 28.160113486510333
32100000 31.16693302577437
32200000 31.715545765120968
32300000 31.809008526876834
32400000 32.87569981431799
32500000 15.896351487372373
32600000 32.67403754854399
32700000 37.407330725434235
32800000 31.238582459251358
32900000 31.299284554760884
33000000 14.898379547214617
33100000 33.07133554635249
332000

59400000 11.389886691360337
59500000 7.175758897480343
59600000 15.596984222280593
59700000 20.621220092745375
59800000 20.165671027120336
59900000 21.58871403702721
60000000 14.082384927714058
60100000 16.03473079012358
60200000 13.800662920828076
60300000 15.08169691057366
60400000 15.234673037465937
60500000 11.33432018158267
60600000 16.42542586657435
60700000 12.596129536451963
60800000 13.522000800789048
60900000 5.214860012659597
61000000 12.559402321620272
61100000 12.145256551386774
61200000 16.017415215306947
61300000 7.412721173286964
61400000 24.592549935982465
61500000 17.552159932672293
61600000 18.21370491708365
61700000 22.26057477232374
61800000 15.063516397271679
61900000 14.074143186926419
62000000 14.926516318748295
62100000 15.961343059552352
62200000 14.126488343563032
62300000 6.130387766899188
62400000 11.777726470359005
62500000 11.817614777192357
62600000 13.070331877766133
62700000 4.935688896707829
62800000 13.317366636986137
62900000 16.52057973493141
63000

89100000 26.942522238083328
89200000 25.413331858411624
89300000 27.07209722247874
89400000 25.101774159494685
89500000 24.972879291315724
89600000 25.43674071840736
89700000 20.804767588326396
89800000 16.94673370702371
89900000 15.139727545183986
90000000 18.912131491191996
90100000 13.23863448096437
90200000 25.008669492094157
90300000 24.032555397984204
90400000 25.736389793246836
90500000 24.334761505073597
90600000 25.08773873168331
90700000 26.122720404204312
90800000 28.549437761648026
90900000 25.71950875895449
91000000 25.98150474492397
91100000 25.045653258607974
91200000 24.306705254932403
91300000 23.671259858946442
91400000 23.881443561621268
91500000 24.439194562121774
91600000 11.348712293021752
91700000 17.442295557889906
91800000 15.03670864179434
91900000 14.01524286697607
92000000 23.392715826364572
92100000 22.939258991203367
92200000 24.47195606658531
92300000 23.563599695967
92400000 24.387778397108036
92500000 24.494047896071393
92600000 26.85087739783588
927000

118200000 26.869784896492074
118300000 24.839903164532828
118400000 24.321603104094415
118500000 26.216023372995586
118600000 25.67674785725165
118700000 23.925579693273516
118800000 24.40976736314639
118900000 15.610125429311557
119000000 3.710074953384657
119100000 16.79954029686756
119200000 24.375543392893746
119300000 24.263749396494013
119400000 24.904569314792333
119500000 24.923587175069038
119600000 24.158614791800854
119700000 25.461650277755503
119800000 26.017922519288376
119900000 25.375186866998995
120000000 20.590217060972833
120100000 21.550595701935745
120200000 19.930697383387326
120300000 21.467489236095194
120400000 20.70748811216282
120500000 20.40017095690441
120600000 19.316309779822884
120700000 14.3377770766339
120800000 3.7543879513826464
120900000 11.448048346861288
121000000 21.170452532490827
121100000 20.136589679317435
121200000 20.639164753224385
121300000 20.55355573334692
121400000 19.851717308592765
121500000 20.817292087585493
121600000 21.6702485311

...

320000000 5.706242985260537
320100000 5.808914881291463
320200000 5.81229027753537
320300000 6.176411996008873
320400000 5.99991360484315
320500000 2.593452871001297
320600000 2.78929393949305
320700000 5.998886438992769
320800000 6.353411512622609
320900000 6.652205190884153
321000000 6.627510280609879
321100000 6.406367092053197
321200000 6.912919123223007
321300000 6.259867796654966
321400000 7.113912153290679
321500000 6.188504102123013
321600000 2.929616845296966
321700000 5.088419272501641
321800000 5.818097821139375
321900000 5.683730061109567
322000000 5.648479681703419
322100000 5.7122170123738645
322200000 5.409391234919081
322300000 3.350034145672772
322400000 1.7421126453253626
322500000 5.968163605355346
322600000 6.0302398923391465
322700000 6.187238843887915
322800000 6.709199676650329
322900000 6.403323091737046
323000000 6.631756919091671
323100000 5.341301165966031
323200000 6.526382011887319
323300000 6.2005413925221955
323400000 2.5632619314282965
323500000 5.634408

348200000 1.9280940440277756
348300000 1.7899883072863851
348400000 2.194243270846251
348500000 1.9131211596818665
348600000 2.9474762034219206
348700000 2.6221773360024923
348800000 4.018895474081506
348900000 1.656689399318791
349000000 1.9908442669789956
349100000 2.24414779797107
349200000 2.6636131997438173
349300000 2.263679776584473
349400000 1.9072834991270502
349500000 2.2347884409163594
349600000 3.5771248763784933
349700000 2.9355561314959826
349800000 1.913964856132342
349900000 1.9586078534736004
350000000 2.2839706332473155
350100000 1.3317074320516982
350200000 1.86651632262617
350300000 1.3664877346298494
350400000 2.6453724210787186
350500000 2.192122632670535
350600000 2.1459205979554254
350700000 1.5719392789575126
350800000 2.5273947515475252
350900000 3.387429905146033
351000000 1.9416280073263412
351100000 3.1471157692415384
351200000 1.365209480882436
351300000 2.669455180192424
351400000 2.57522710063279
351500000 3.4062434741666814
351600000 1.8218512300210097


376700000 1.8104831788082223
376800000 1.433707067270617
376900000 1.4865189862986505
377000000 1.3845172081165016
377100000 0.9167302477536731
377200000 2.214039632087353
377300000 1.31182424957636
377400000 1.8203856608676048
377500000 1.081757378619651
377600000 1.835680243447711
377700000 2.4784768560609383
377800000 2.022455863341994
377900000 1.758839570243776
378000000 0.9935748844048545
378100000 2.4461544811817806
378200000 1.4127396298005583
378300000 1.685149554299669
378400000 1.0691027574521799
378500000 1.6991893319406908
378600000 2.4923245364714663
378700000 1.3858399467770424
378800000 2.2015451332363485
378900000 0.9473007134773134
379000000 1.3246978042459754
379100000 1.1854381143588721
379200000 1.4365044097540478
379300000 1.6284013668034076
379400000 1.8019524268184601
379500000 2.4073191872215505
379600000 2.2609549974790446
379700000 2.577417475428164
379800000 1.6096592166340316
379900000 1.5864365422096585
380000000 1.3430367892572876
380100000 1.399682310887

404000000 0.4516641384730339
404100000 0.7336185452440579
404200000 0.7276473500399888
404300000 0.5520337265796462
404400000 0.45018722595694993
404500000 0.5276151971758472
404600000 0.7256624416671792
404700000 0.6282157946004536
404800000 0.47156509974072375
404900000 0.38566026706054807
405000000 0.6180004737100601
405100000 0.6755511971000301
405200000 0.6175340023375651
405300000 0.3095327662829459
405400000 0.5804021363574432
405500000 0.6148743630443017
405600000 0.565872023756322
405700000 0.4507173165997995
405800000 0.31054763543638586
405900000 0.5459191852511895
406000000 0.6076589521448943
406100000 0.5533931125367979
406200000 0.2846149414558411
406300000 0.5124041638076008
406400000 0.5904523704448343
406500000 0.6631814037644167
406600000 0.5834504770382478
406700000 0.34664387424516674
406800000 0.603830559683442
406900000 0.416227069188873
407000000 0.10867116777439913
407100000 0.10797152897803321
407200000 0.10789558566452066
407300000 0.10761988113658627
40740000

430200000 0.0656197482913269
430300000 0.0916219957147671
430400000 0.05961791959085068
430500000 0.08649911621377203
430600000 0.06456723853719565
430700000 0.07808207960567871
430800000 0.07567707890372805
430900000 0.06127454521574577
431000000 0.08011838201200962
431100000 0.053748674731373784
431200000 0.07680537726615866
431300000 0.05386663112392029
431400000 0.07199741100793085
431500000 0.05616689660889573
431600000 0.06434456785393754
431700000 0.06530663794740703
431800000 0.052895406542470054
431900000 0.06712969218298794
432000000 0.04733968773872654
432100000 0.06692556871728765
432200000 0.0483244873726666
432300000 0.05674887311169505
432400000 0.050443119915260216
432500000 0.05048381015223265
432600000 0.05498962226688862
432700000 0.04245940954216984
432800000 0.05333039506807923
432900000 0.04319436991445555
433000000 0.04871343255193034
433100000 0.03750048816275597
433200000 0.04451623789494899
433300000 0.041960035015698935
433400000 0.03447934901183843
433500000